In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
# from matrix_ops_wrapper import add_matrices, matmul_simple

In [2]:
import os
current_dir = os.getcwd()
print(f"{current_dir = }")
os.chdir('/home/masroor/projects/quantumgrad/')

current_dir = '/home/masroor/projects/quantumgrad/notebooks'


In [12]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

def get_train_loader():
    trainset = datasets.MNIST(root='./data', train=True, download=False, transform=transform)
    trainloader = DataLoader(trainset, batch_size=256, shuffle=True)
    return trainloader

def get_test_loader():
    testset = datasets.MNIST(root='./data', train=False, download=False, transform=transform)
    testloader = DataLoader(testset, batch_size=256, shuffle=False)
    return testloader

## PyTorch

In [4]:
class TorchMNIST(nn.Module):
    def __init__(self):
        super(TorchMNIST, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"device is {device}")

device is cuda


In [13]:
# Move the entire dataset to the CUDA device
trainloader = get_train_loader()
testloader = get_test_loader()
trainloader = [(images.to(device), labels.to(device)) for images, labels in trainloader]
testloader = [(images.to(device), labels.to(device)) for images, labels in testloader]

In [14]:
torch_model = TorchMNIST().to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(torch_model.parameters(), lr=0.01, momentum=0.9)

In [15]:
epochs = 5
for epoch in range(epochs):
    torch_model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        optimizer.zero_grad()
        output = torch_model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, loss: {running_loss/len(trainloader)}")
    

Epoch 1, loss: 0.6215460312493304
Epoch 2, loss: 0.2298121525886211
Epoch 3, loss: 0.16673300446982078
Epoch 4, loss: 0.1305255803339025
Epoch 5, loss: 0.1067689300851619


In [18]:
torch_model.eval()
correct = 0
total = 0
test_loss = 0.0
with torch.no_grad():
    for images, labels in testloader:
        output = torch_model(images)
        test_loss += criterion(output, labels).item()
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test loss: {test_loss/len(testloader)}")
print(f"Test accuracy: {100 * correct / total}%")

Test loss: 0.09989477843046188
Test accuracy: 96.81%


## Quantumgrad (oh wait it doesn't have anything lmao)